Student Name: Joel Amarou Heuer

Student ID: 202102201

# Imports

In [1]:
import numpy as np
import pandas as pd
import random
import math     

# for tokenization use nltk
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /home/korosu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# (Hyper)Parameters

In [2]:
# TODO Hyperparameters
# input_size = 
output_size = 10 # -> number of neurons in output-layer (= for each possible rating?)
# [hidden_layers_sizes] = 
# learning_rate = 
number_of_epochs = 1 # how is every sample used for pass/forward/backward


# Parameters
train_data_path = "./data/drugLibTrain_raw.tsv" # please use relative path like this
test_data_path = "./data/drugLibTest_raw.tsv" # please use relative path like this

# Main

In [3]:
input_features =  'commentsReview'

###########################################
################ Load Data ################
###########################################
# load data
train_data = pd.read_csv(train_data_path, sep='\t')
test_data = pd.read_csv(test_data_path, sep='\t')
# drop nans
train_data.dropna(subset = [input_features], inplace=True)
test_data.dropna(subset = [input_features], inplace=True)
# lowercase all comments
train_data[input_features] = train_data[input_features].str.lower()
test_data[input_features] = test_data[input_features].str.lower()

In [4]:
############################################################
################ Create directory for words ################
############################################################
# merge data
df_data =  pd.concat([train_data,test_data], axis=0)
# create dictonary
dictonary_words = df_data["commentsReview"].apply(nltk.word_tokenize)
dictonary_words = dictonary_words.values.tolist()
# flat lists
dictonary_words = [item for sublist in dictonary_words for item in sublist]
# drop duplicates
dictonary_words = list(set(dictonary_words))
# number of all words
dict_size = len(dictonary_words)

print(f"Number of words {dict_size}")

Number of words 11584


In [5]:
##############################################
################ Tokenization ################
##############################################
train_data_tokenized:pd.DataFrame = train_data["commentsReview"].apply(nltk.word_tokenize)
test_data_tokenized:pd.DataFrame = test_data["commentsReview"].apply(nltk.word_tokenize)

In [6]:
##############################################################
################ Word Encoding (Bag of words) ################
##############################################################
def to_bow(dataset):
    '''
    @param dataset which should be considered (train or test)
    @return new column for dataset with sentence/word-encoded cells

    For each cell in column of interest, following is performed:
    (1) cell contains sentence
    (2) sentence is tokenized --> produces array of words/tokens
    (3) create array A with len(dict_size) 
    (4) for each token count occurance 
    (5) A[index of token t in dict] = occurance of t
    '''
    new_col = []

    # for each row
    for _, row in dataset.iterrows():
        tokenized = nltk.word_tokenize(row[input_features])
        array = np.zeros(shape=dict_size)

        # for each word in a setence
        for word in tokenized:
            i_hot = dictonary_words.index(word)
            array[i_hot] = array[i_hot] + 1
        new_col.append(array)
    
    return new_col


test_data[input_features] =  to_bow(test_data)
train_data[input_features] =  to_bow(train_data)
train_data.sample(2)

,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
1540,3248,nuvaring,3,Highly Effective,Extremely Severe Side Effects,i kept getting pregnant. no more stairs.... pl...,Never got pregnant,My dosage was so high that at the end of a wee...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2221,4123,chantix,7,Highly Effective,Moderate Side Effects,nicotine addiction,I had smoked for 20 years and despite repeated...,I was nauseated after taking each tablet. The ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [38]:
#################################################
################ Create datasets ################
#################################################
# split into train & test    and    feature & y columns
train_x = train_data[input_features]
train_y = train_data['rating']
test_x = test_data[input_features]
test_y = test_data['rating']

#####################################################################
################ # ONE-HOT for output (classes 1-10) ################
#####################################################################
# creating one-hot vector notation of labels. 
# (Labels are given numeric in the dataset)
new_train_y = np.zeros(shape=(len(train_y), output_size))
new_test_y = np.zeros(shape=(len(test_y), output_size))

# add 1's into one-hot-matrix
for i in range(len(test_y)):
    new_test_y[i][test_y.iloc[i]-1] = 1

for i in range(len(train_y)) :    
    new_train_y[i][train_y.iloc[i]-1] = 1

train_y = new_train_y
test_y = new_test_y


##################################################################
################ split into validation & training ################
##################################################################
# Split training data into [a] Training (75%) and [b] validation (25%)
valid_x = np.asarray(train_x[int(0.75*len(train_x)):-1])
valid_y = np.asarray(train_y[int(0.75*len(train_y)):-1])
train_x = np.asarray(train_x[0:int(0.75*len(train_x))])
train_y = np.asarray(train_y[0:int(0.75*len(train_y))])




'''WIP'''
# Hyperparamater
num_neurons_in_h1 = 250
num_neurons_in_h2 = 200

w_b = {
    # Input-Layer -> H1
    "L0_w" : np.random.randn(num_neurons_in_h1, dict_size), 
    "L0_b": np.ones((num_neurons_in_h1, 1)) * 0.01,

    # H1 -> H2
    "L1_w" : np.random.randn(num_neurons_in_h2, num_neurons_in_h1), # WIP
    "L1_b": np.ones((num_neurons_in_h2, 1)) * 0.01,

    # H2 -> Output-Layer
    "L2_w" : np.random.randn(output_size, num_neurons_in_h2), # WIP
    "L2_b": np.ones((output_size, 1)) * 0.01,

}





#######################################
################ train ################
#######################################
# TODO
train(train_x, train_y, valid_x, valid_y)

######################################
################ test ################
######################################
# TODO 
#print("Test Scores:")
#print(test(test_x, test_y))

[[10.42169984  4.29525646  0.          6.24997091  7.01504583  5.12781748
   0.          9.94769998  0.         11.26922242]]

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [33]:
def activation_function(z):
	'''
	sigmoid-function is used as activation function for hidden-layers
	'''
	return 1/(1 + np.exp(-z))

def reLU(z):
    '''
	reLU is used as activation for output-layer
	'''
    return np.maximum(0.0, z)


def forward_pass(data):
	# Input --> Hidden-1
	z0 = np.dot(data, w_b['L0_w'].T) + w_b['L0_b'].T
	a0 = activation_function(z0)

	# Hidden-1 --> Hidden-2
	z1 = np.dot(a0, w_b['L1_w'].T) + w_b['L1_b'].T
	a1 = activation_function(z1)


	# LayerHidden2 --> LayerOutput
	z2 = np.dot(a1, w_b['L2_w'].T) + w_b['L2_b'].T
	y_pred = reLU(z2)

	results={
		"a0": a0,
		"a1": a1,
		"y_pred": y_pred,
	}
	return results

def train(train_data, train_labels, valid_data, valid_labels):

	for epoch in range(number_of_epochs):
		index = 0

		# Same thing about [hidden_layers] mentioned above is valid here also
		for data, labels in zip(train_data, train_labels):
			predictions = forward_pass(data)
			break
#			loss_signals = derivation_of_loss_function(labels, predictions)
#			backward_pass(data, [hidden_layers], predictions, loss_signals)
#			loss = loss_function(labels, predictions)
#
#			if index % 400 == 0: # at each 400th sample, we run validation set to see our model's improvements
#				accuracy, loss = test(valid_data, valid_labels)
#				print("Epoch= "+str(epoch)+", Coverage= %"+ str(100*(index/len(train_data))) + ", Accuracy= "+ str(accuracy) + ", Loss= " + str(loss))
#
#			index += 1
#
#	return losses